# Setting the Dataset up

In [1]:
#Importing necessary libraries 
import pandas as pd
import sqlite3

In [2]:
#collecting wikipedia data as a data frame
data=pd.read_html('https://en.wikipedia.org/wiki/List_of_companies_of_India#Largest_firms', header=0)

In [54]:
#viewing the collected data
#The following data gives the details about major corporations in India
data[1]

,Name,Industry,Sector,Headquarters,Founded,Notes,Status,Status.1,Ownership_Status,Present_Status
0,Name,Industry,Sector,Headquarters,Founded,Notes,NaN,NaN,NaN,NaN
1,63 Moons Technologies,Technology,Software,Mumbai,1988,Financial technology,P,A,P,A
2,Aban Offshore,Energy,Oil & gas offshore drilling & other services,Chennai,1986,"Oil, petrochemical",P,A,P,A
3,ABG Shipyard,Industrials,Shipbuilding,Ahmedabad,1985,Ship building,P,A,P,A
4,ABP Group,Consumer services,Broadcasting & entertainment,Kolkata,1922,"Media, news",P,A,P,A
...,...,...,...,...,...,...,...,...,...,...
474,Zensar Technologies,Technology,Software & IT services,Pune,1991,IT services and software,P,A,P,A
475,Zeta,Technology,Consumer digital services,Mumbai,2015,Fintech,P,A,P,A
476,Zoho Corporation,Technology,Software,Chennai,1996,Business management software,P,A,P,A
477,Zomato,Technology,Consumer digital services,Gurugram,2008,Food delivery,P,A,P,A


In [4]:
#creating a database with the data
df=data[1]

In [5]:
df.columns

Index(['Name', 'Industry', 'Sector', 'Headquarters', 'Founded', 'Notes',
       'Status', 'Status.1'],
      dtype='object')

In [29]:
#Renaming the column names 
df['Ownership_Status'] = df['Status']
df['Present_Status'] = df['Status.1']

In [7]:
#removing the first row, which is not required
df_c = df.drop(index=0, columns=["Status", "Status.1"])
df_c

,Name,Industry,Sector,Headquarters,Founded,Notes,Ownership_Status,Present_Status
1,63 Moons Technologies,Technology,Software,Mumbai,1988,Financial technology,P,A
2,Aban Offshore,Energy,Oil & gas offshore drilling & other services,Chennai,1986,"Oil, petrochemical",P,A
3,ABG Shipyard,Industrials,Shipbuilding,Ahmedabad,1985,Ship building,P,A
4,ABP Group,Consumer services,Broadcasting & entertainment,Kolkata,1922,"Media, news",P,A
5,ACC,Industrials,Cement,Mumbai,1936,Cement,P,A
...,...,...,...,...,...,...,...,...
474,Zensar Technologies,Technology,Software & IT services,Pune,1991,IT services and software,P,A
475,Zeta,Technology,Consumer digital services,Mumbai,2015,Fintech,P,A
476,Zoho Corporation,Technology,Software,Chennai,1996,Business management software,P,A
477,Zomato,Technology,Consumer digital services,Gurugram,2008,Food delivery,P,A


In [8]:
df_c.columns

Index(['Name', 'Industry', 'Sector', 'Headquarters', 'Founded', 'Notes',
       'Ownership_Status', 'Present_Status'],
      dtype='object')

In [9]:
#Creating a connection
conn=sqlite3.connect('companies.db')
c=conn.cursor()

In [30]:
#Creating the table
c.execute(
    """ CREATE TABLE IF NOT EXISTS company('Name' text, 'Industry' text, 'Sector' text, 'Headquarters' text, 
    'Founded' int, 'Notes' text,'Ownership_Status' text, 'Present_Status' text);""")

In [11]:
#Adding data to table
df_c.to_sql('company', conn, if_exists='replace', index=False)

478

In [12]:
#Creating a wrapper function
def rsql(query):
    return pd.read_sql_query(query, conn)

## Using WHERE, LIMIT

In [31]:
rsql('SELECT * from company where "Industry" = "Technology" LIMIT 5')

,Name,Industry,Sector,Headquarters,Founded,Notes,Ownership_Status,Present_Status
0,63 Moons Technologies,Technology,Software,Bangalore,1988,Financial technology,P,A
1,BluSmart,Technology,Consumer digital services,Gurugram,2019,"Taxi services, ride-sharing",P,A
2,Cosmic Circuits,Technology,Semiconductors,Bengaluru,2005,Acquired by Cadence Design Systems (USA),P,D
3,Cyient,Technology,Software & IT services,Hyderabad,1991,"Engineering outsourcing services, software man...",P,A
4,Flipkart,Technology,Consumer digital services,Bengaluru,2007,E-commerce,P,A


## Using AS, COUNT, GROUP BY, ORDER BY, DESC, LIMIT

In [14]:
#Finding Top 5 cities with the most headquarters
rsql("""SELECT "Headquarters" as "City", COUNT("Headquarter") as "Number_of_HQs" 
from company group by "Headquarters" 
order by count("headquarters") DESC LIMIT 5""")

,City,Number_of_HQs
0,Mumbai,145
1,New Delhi,62
2,Bengaluru,45
3,Chennai,30
4,Kolkata,28


In [22]:
#Count of companies by Industries
rsql("SELECT Industry, COUNT(*) AS Company_Count FROM company GROUP BY Industry ORDER BY Company_Count DESC LIMIT 5")

,Industry,Company_Count
0,Industrials,109
1,Consumer goods,68
2,Financials,51
3,Conglomerate,44
4,Technology,38


In [23]:
#Finding count of companies by ownership status
rsql("SELECT Ownership_Status, COUNT(*) AS Total_Companies FROM company GROUP BY Ownership_Status")

,Ownership_Status,Total_Companies
0,P,378
1,S,100


## USING AVG

In [15]:
#Finding the average age of Companies by Industry
rsql(""" SELECT Industry, Avg(2025-"Founded") as "Avg_Age_of_Companies" from company group by "Industry" """)

,Industry,Avg_Age_of_Companies
0,Basic materials,64.678571
1,Chemicals,56.470588
2,Conglomerate,131.340909
3,Consumer goods,86.073529
4,Consumer services,43.484848
5,Energy,46.733333
6,Financials,69.784314
7,Health care,126.821429
8,Industrials,50.944954
9,Insurance,16.000000


## USING BETWEEN, AND

In [35]:
#Companies founded between 2000 and 2010 in Technology Industry 
rsql("""SELECT COUNT(*) AS Tech_Companies_From_2000_to_2010 
FROM company WHERE Founded between 2000 AND 2010
AND Industry = 'Technology'""")

,Tech_Companies_From_2000_to_2010
0,13


## Using HAVING

In [24]:
#Finding industries with more than 10 companies using HAVING FUNCTION
rsql("SELECT Industry, COUNT(*) as Company_Count FROM company GROUP BY Industry HAVING COUNT(*) > 10")

,Industry,Company_Count
0,Basic materials,28
1,Chemicals,17
2,Conglomerate,44
3,Consumer goods,68
4,Consumer services,33
5,Energy,15
6,Financials,51
7,Health care,28
8,Industrials,109
9,Technology,38


In [28]:
#Finding sectors with average founded year after 2000
rsql("SELECT Sector, AVG(Founded) as Avg_Founded_Year, COUNT(*) as Company_Count FROM company GROUP BY Sector HAVING AVG(Founded) > 2000")

,Sector,Avg_Founded_Year,Company_Count
0,Aerospace,2003.00,1
1,Alternative energy,2010.00,1
2,Consumer digital services,2005.12,25
3,Full line insurance,2009.00,2
4,Semiconductors,2005.00,1
5,Travel & tourism,2007.00,2


## Using LIKE

In [25]:
#Searcing companies by name pattern using LIKE FUNCTION
rsql("SELECT Name, Industry, Headquarters FROM company WHERE Name LIKE '%Tech%'")

,Name,Industry,Headquarters
0,63 Moons Technologies,Technology,Mumbai
1,Aptech,Industrials,Mumbai
2,Bharat Biotech,Health care,Hyderabad
3,Dixon Technologies,Industrials,Noida
4,Dynamatic Technologies,Industrials,Bengaluru
5,HCLTech,Industrials,Noida
6,Intex Technologies,Consumer goods,New Delhi
7,KPIT Technologies,Consumer goods,Pune
8,Lanco Infratech,Conglomerate,New Delhi
9,Protean eGov Technologies,Technology,Mumbai


## Using INSERT

In [59]:
#Inserting New Company into the data using INSERT FUNCTION
c.execute("""INSERT INTO company (Name, Industry, Sector, Headquarters, Founded, Notes, Ownership_Status, Present_Status) 
             VALUES ('New Tech Corp', 'Technology', 'Software', 'Bangalore', 2023, 'AI startup', 'P', 'A')""")
rsql("SELECT * from company WHERE Name = 'New Tech Corp'")

,Name,Industry,Sector,Headquarters,Founded,Notes,Ownership_Status,Present_Status
0,New Tech Corp,Technology,Software,Bangalore,2023,AI startup,P,A
1,New Tech Corp,Technology,Software,Bangalore,2023,AI startup,P,A


## Using UPDATE

In [27]:
# Updating company status
c.execute("""UPDATE company SET Headquarters = 'Bangalore' WHERE Name = '63 Moons Technologies'""")
rsql('SELECT * from company where "Industry" = "Technology" LIMIT 5')

,Name,Industry,Sector,Headquarters,Founded,Notes,Ownership_Status,Present_Status
0,63 Moons Technologies,Technology,Software,Bangalore,1988,Financial technology,P,A
1,BluSmart,Technology,Consumer digital services,Gurugram,2019,"Taxi services, ride-sharing",P,A
2,Cosmic Circuits,Technology,Semiconductors,Bengaluru,2005,Acquired by Cadence Design Systems (USA),P,D
3,Cyient,Technology,Software & IT services,Hyderabad,1991,"Engineering outsourcing services, software man...",P,A
4,Flipkart,Technology,Consumer digital services,Bengaluru,2007,E-commerce,P,A


## Using DISTINCT

In [52]:
#Finding the different headquarter cities
rsql(" SELECT DISTINCT(Headquarters) from company")

,Headquarters
0,Mumbai
1,Chennai
2,Ahmedabad
3,Kolkata
4,Faridabad
5,New Delhi
6,Kanpur
7,Vijayawada
8,Kochi
9,Gurugram


In [49]:
#Finding the years with the most number of companies founded
rsql(""" SELECT Distinct(Founded) as Founding_Year, Count(Name) as Number_of_Companies_Founded 
from company GROUP BY Founded 
ORDER BY Count(Name)  DESC """)

,Founding_Year,Number_of_Companies_Founded
0,1986,12
1,1995,11
2,1983,9
3,1981,9
4,1979,9
...,...,...
126,1806,1
127,1788,1
128,1775,1
129,1767,1


In [53]:
#Given the nature of the dataset, we did not find any suitable use of the 'SUM' function